<b>ASSESSMENT PART 1 - Transform the data on the Wikipedia page into a pandas dataframe</b>

Import all necessary modules:

In [9]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

!pip install geocoder
import geocoder

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from sklearn.metrics import silhouette_score

Get data & soup from Wikipedia:

In [25]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url)
soup = BeautifulSoup(html.text, 'lxml')

Scrape table and clean data:

In [26]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

Convert to Dataframe:

In [29]:
df = pd.DataFrame(table_contents)

In [32]:
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df = df.rename(columns={'PostalCode':'Postal Code'}) # Rename Column to match next dataset

In [96]:
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [35]:
df.shape

(103, 3)

<b>ASSESSMENT PART 2 - Get the latitude and the longitude coordinates of each neighborhood</b>

Download data from CSV:

In [54]:
lldataurl = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv'
lldata = requests.get(lldataurl)
lldf = pd.read_csv(lldataurl)


In [57]:
lldf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Combine into single DataFrame:

In [58]:
dfCombined = pd.merge(df,lldf, on='Postal Code')
dfCombined.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


<b>ASSESSMENT PART 3 - Explore and cluster the neighborhoods in Toronto</b>

Find Lat/Long of Toronto:

In [ ]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

Create map of Toronto using latitude and longitude values


In [100]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(dfCombined['Latitude'], dfCombined['Longitude'], dfCombined['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Calculate Silouhette Score to find best value of K


In [102]:
sil = []
kmax = 10

# dissimilarity would not be defined for a single cluster, thus, minimum number of clusters should be 2
for k in range(2, kmax+1):
  kmeans = KMeans(n_clusters = k).fit(dfCombined[['Latitude','Longitude']])
  labels = kmeans.labels_
  sil.append(silhouette_score(dfCombined[['Latitude','Longitude']], labels, metric = 'euclidean'))

sil

[0.4153022281040723,
 0.42581728117187045,
 0.40195837781212757,
 0.36900061636882053,
 0.37661438457153956,
 0.37551498849236004,
 0.36674734562354244,
 0.36359786496080027,
 0.36411602491823364]

Apply Kmeans clustering with optimum value of k (k=3)

In [86]:
# set number of clusters
kclusters = 3

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dfCombined[['Latitude','Longitude']])

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 1, 2, 1, 0, 2, 0, 2], dtype=int32)

In [94]:
# add clustering labels
dfCombined.insert(0, 'Cluster Labels', kmeans.labels_)

dfCombined.head()

,Cluster Labels,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.753259,-79.329656
1,0,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,1,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,2,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


Create cluster map of Toronto using latitude and longitude values


In [95]:
cluster_map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
colors = ['red','blue','gray','orange', 'beige','green','purple', 'cadetblue', 'black','pink']
# add markers to map
for cll, lat, lng, label in zip(dfCombined['Cluster Labels'], dfCombined['Latitude'], dfCombined['Longitude'], dfCombined['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=colors[cll],
        fill=True,
        fill_color=colors[cll-1],
        fill_opacity=0.7,
        parse_html=False).add_to(cluster_map_toronto)  
    
cluster_map_toronto